## (ビジュアル) チャット補完推論のオンラインエンドポイントの使用

このサンプルでは、`Phi-3-vision-128k-instruct` をオンラインエンドポイントにデプロイして推論を行う方法を示します。

### 概要
* 前提条件の設定
* デプロイするモデルの選択
* 推論用データのダウンロードと準備
* モデルのデプロイによるリアルタイム推論
* エンドポイントのテスト
* Azure OpenAI スタイルのペイロードを使用したエンドポイントのテスト
* リソースのクリーンアップ

### 1. 前提条件の設定
* 依存関係のインストール
* AzureML ワークスペースに接続します。詳細は [SDK 認証の設定](https://learn.microsoft.com/azure/machine-learning/how-to-setup-authentication?tabs=sdk) を参照してください。以下の `<WORKSPACE_NAME>`、`<RESOURCE_GROUP>`、および `<SUBSCRIPTION_ID>` を置き換えます。
* `azureml` システムレジストリに接続します

In [ ]:
# 必要なモジュールをインポートします
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)

try:
    # デフォルトの Azure 資格情報を取得しようとします
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # デフォルトの資格情報が利用できない場合、インタラクティブブラウザ資格情報を使用します
    credential = InteractiveBrowserCredential()

try:
    # 提供された資格情報を使用して MLClient を作成しようとします
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    resource_group = workspace_ml_client.resource_group_name
    workspace_name = workspace_ml_client.workspace_name
except Exception as ex:
    print(ex)
    # MLClient の作成に失敗した場合、AML ワークスペースの詳細を手動で入力します
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<WORKSPACE_NAME>"

# 提供された資格情報とワークスペースの詳細を使用して MLClient インスタンスを作成します
workspace_ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

# モデル、ファインチューニングパイプライン、および環境は AzureML システムレジストリ "azureml" にあります
registry_ml_client = MLClient(credential, registry_name="azureml")

### 2. モデルをオンラインエンドポイントにデプロイする
オンラインエンドポイントは、モデルを使用する必要があるアプリケーションと統合するために使用できる耐久性のある REST API を提供します。

In [ ]:
# このコードは、指定された名前のモデルがレジストリに存在するかどうかを確認します。
# モデルが存在する場合、モデルの最初のバージョンを取得し、その詳細を表示します。
# モデルが存在しない場合、モデルが見つからなかったことを示すメッセージを表示します。

# model_name: レジストリで確認するモデルの名前
model_name = "Phi-3-vision-128k-instruct"

# 指定されたモデル名のバージョンリストを取得します
version_list = list(registry_ml_client.models.list(model_name))

# レジストリにモデルのバージョンが存在するかどうかを確認します
if len(version_list) == 0:
    print("レジストリにモデルが見つかりません")
else:
    # モデルの最初のバージョンを取得します
    model_version = version_list[0].version
    foundation_model = registry_ml_client.models.get(model_name, model_version)
    
    # モデルの詳細を表示します
    print(
        "\n\n推論に使用するモデル名: {0}, バージョン: {1}, ID: {2}".format(
            foundation_model.name, foundation_model.version, foundation_model.id
        )
    )

In [ ]:
# 必要なモジュールをインポートします
import time
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

# オンラインエンドポイントを作成します - エンドポイント名はリージョン内で一意である必要があるため、タイムスタンプを使用して一意のエンドポイント名を作成します
timestamp = int(time.time())
online_endpoint_name = model_name[:13] + str(timestamp)
print(f"名前: {online_endpoint_name} のオンラインエンドポイントを作成します")

# オンラインエンドポイントを作成します
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description=f"{foundation_model.name} のオンラインエンドポイント、ビジュアルチャット補完タスク用",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
# このコードは、オンラインエンドポイントのデプロイメントを作成します。
# デプロイメント名、エンドポイント名、モデル、インスタンスタイプ、インスタンス数、およびリクエスト設定を設定します。
# また、ライブネスプローブとレディネスプローブの設定も行います。
# 最後に、エンドポイントのトラフィック分布を更新します。

from azure.ai.ml.entities import OnlineRequestSettings, ProbeSettings

# デプロイメントを作成します
deployment_name = "phi-3-vision"
demo_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_NC48ads_A100_v4",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=180000,
        max_queue_wait_ms=500,
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=49,
        success_threshold=1,
        timeout=299,
        period=180,
        initial_delay=180,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=10,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {deployment_name: 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

### 3. サンプルデータを使用してエンドポイントをテストする

以下に作成する json を使用して、モデルにサンプルリクエストを送信します。

In [ ]:
# 必要なモジュールをインポートします
import json
import os

# テスト JSON ペイロードを定義します
test_json = {
    "input_data": {
        "input_string": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": "https://www.ilankelman.org/stopsigns/australia.jpg"
                        },
                    },
                    {
                        "type": "text",
                        "text": "この画像に何が表示されていますか？非常に詳細かつ具体的に説明してください。",
                    },
                ],
            },
        ],
        "parameters": {"temperature": 0.7, "max_new_tokens": 2048},
    }
}

# JSON オブジェクトをファイルに保存します
sample_score_file_path = os.path.join(".", "sample_chat_completions_score.json")
with open(sample_score_file_path, "w") as f:
    json.dump(test_json, f, indent=4)

# 入力ペイロードを表示します
print("入力ペイロード:\n")
print(test_json)

In [ ]:
# 必要なモジュールをインポートします
import pandas as pd

# azureml エンドポイントの invoke メソッドを使用して、sample_chat_completions_score.json ファイルをオンラインエンドポイントでスコアリングします
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment_name,
    request_file=sample_score_file_path,
)
print("生の JSON 応答: \n", response, "\n")

# JSON 文字列を解析します
json_data = json.loads(response)

# 解析された JSON を DataFrame に変換します
response_df = pd.DataFrame([json_data])
print("生成されたテキスト:\n", response_df["output"].iloc[0])

### 4. Azure OpenAI スタイルのペイロードを使用してエンドポイントをテストする

Azure OpenAI スタイルのペイロードを使用して、モデルにサンプルリクエストを送信します。

In [ ]:
# このコードは、Azure OpenAI スタイルのペイロードを使用してオンラインエンドポイントをテストするための JSON ペイロードを定義します。
# これには、モデル名、ユーザーロールとコンテンツ（画像 URL とテキスト）を含むメッセージのリスト、
# 温度、および max_new_tokens が含まれます。

aoai_test_json = {
    "model": foundation_model.name,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://www.ilankelman.org/stopsigns/australia.jpg"
                    },
                },
                {
                    "type": "text",
                    "text": "この画像に何が表示されていますか？非常に詳細かつ具体的に説明してください。",
                },
            ],
        }
    ],
    "temperature": 0.7,
    "max_new_tokens": 2048,
}

In [ ]:
# スコアリング URI を取得します
scoring_uri = workspace_ml_client.online_endpoints.get(
    name=online_endpoint_name
).scoring_uri
# AOAI 用にスコアリング URI を更新します
aoai_format_scoring_uri = scoring_uri.replace("/score", "/v1/chat/completions")

# データプレーン操作のキーを取得します
data_plane_token = workspace_ml_client.online_endpoints.get_keys(
    name=online_endpoint_name
).primary_key

In [ ]:
import urllib.request
import json

# リクエストを準備します
body = str.encode(json.dumps(aoai_test_json))
url = aoai_format_scoring_uri
api_key = data_plane_token

headers = {"Content-Type": "application/json", "Authorization": ("Bearer " + api_key)}
req = urllib.request.Request(url, body, headers)

# リクエストを送信し、応答を取得します
try:
    response = urllib.request.urlopen(req)
    result = response.read().decode("utf-8")
    print(result)
except urllib.error.HTTPError as error:
    print("リクエストがステータスコード: " + str(error.code) + " で失敗しました")
    # ヘッダーを表示します - これにはリクエスト ID とタイムスタンプが含まれており、失敗のデバッグに役立ちます
    print(error.info())
    print(error.read().decode("utf8", "ignore"))

### 5. オンラインエンドポイントを削除する
オンラインエンドポイントを削除することを忘れないでください。そうしないと、エンドポイントが使用する計算リソースの料金が発生し続けます。

In [ ]:
# ワークスペースを削除します
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()